In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
products = pd.read_csv('product_titles.csv',encoding ='latin-1')

In [19]:
products['Product_Name']

0      Offres Privées : Pack d'1 Livre Photo Luxe jus...
1      Groupon - A4 Personalised Photobook Credit (up...
2             Pack 1 Livre Photo Luxe 100 pages (1 mois)
3      2 x A4 Classic Photobook with 60 Pages, Deluxe...
4      2 x A4 Classic Collage Book with 100 Pages (3 ...
5      Livre Photo Prestige A4 jusqu'à 160 pages + co...
6      Livre Photo Portrait A4 50 pages + Option Couv...
7      Offres Privées : Pack de 2 Livres Photo Presti...
8                Livre Photo Mosaïque - jusqu'à 90 pages
9      Square Hardcover Photobook with 60 pages, Prem...
10     Offres Privées : Pack d'1 Livre Photo Maman de...
11     1 x A4 Personalised Photobook with 26 Pages (6...
12     Pack 1 Livre Photo Prestige A4 jusqu'à 120 pag...
13     A4 Personalised Photo Book & A4 Classic PhotoB...
14     A3 Personalised Photobook with 100 Pages and L...
15         Guthaben: Fotobuch Prestige XXL A3, 60 Seiten
16     Large Square Hardcover Photobook with 50 Pages...
17     Oferty VIP : Jednostki n

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tfidf = TfidfVectorizer(stop_words='english')

In [7]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(products['Product_Name'])

In [8]:
tfidf_matrix.shape

(1000, 356)

In [9]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
indices = pd.Series(products.index, index=products['Product_Name']).drop_duplicates()

In [11]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    prod_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return products['Product_Name'].iloc[prod_indices]

In [20]:
get_recommendations('A3 Personalised Photobook 100 Pages')

885    2 x A3 Personalised Photobook with 100 Pages (...
165    A3 Personalised Photobook with 100 Pages + No ...
156    A3 Personalised Photobook with 100 Pages and L...
807                   A3 Personalised Photobook 26 pages
766                                         Photobook A3
32     1 x A3 Personalised Photobook with 50 Pages (3...
220    2 x A3 Personalised Photobook with 60 Pages (1...
888    A3 Personalised Photobook with 100 Pages and D...
219    A3 Personalised Photobook with 50 Pages + No Logo
590    A3 Personalised Photobook with 120 Pages (3 Mo...
Name: Product_Name, dtype: object

In [21]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
Product_Name    1000 non-null object
category        1000 non-null object
size            1000 non-null object
dtypes: object(3)
memory usage: 23.5+ KB


In [30]:
def create_soup(x):
    return ' '.join(x['Product_Name']) + ' ' + ' '.join(x['category'])+ ' ' + ' '.join(x['size'])

In [31]:
# Create a new soup feature
products['soup'] = products.apply(create_soup, axis=1)

In [50]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

In [52]:
count_matrix = count.fit_transform(products['soup'])

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [65]:
def make_list(x):
    return ''.join(x['Product_Name']) +''+''.join(x['category']) +''+''.join(x['size'])

In [66]:
products['concat'] = products.apply(make_list, axis=1)

In [68]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

In [69]:
count_matrix = count.fit_transform(products['concat'])

In [71]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(products['concat'])
type(count_matrix)

scipy.sparse.csr.csr_matrix

In [72]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [73]:
# Reset index of your main DataFrame and construct reverse mapping as before
products = products.reset_index()
indices = pd.Series(products.index, index=products['Product_Name'])

In [75]:
get_recommendations('A3 Personalised Photobook 26 pages', cosine_sim2)

285                   A3 Personalised Photobook 42 pages
977                  A3 Personalised Photobook 100 Pages
42     2 X A3 Personalised Photobook with 26 Pages (3...
140                   A4 Personalised Photobook 26 pages
645              A4 Personalised Photobook with 26 pages
158    2 X A3 Personalised Photobook with 26 Pages +M...
165    A3 Personalised Photobook with 100 Pages + No ...
219    A3 Personalised Photobook with 50 Pages + No Logo
579    A3 Personalised Photobook with 32 Pages + No Logo
128    1 x A3 Personalised Photobook with 26 Pages an...
Name: Product_Name, dtype: object